In [1]:
import time

import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split


def train_model():
    """Train model helper function."""
    X, y = make_classification(random_state=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    tic = time.time()
    clf = LogisticRegression(random_state=0)
    grid = GridSearchCV(
        clf, param_grid={"C": np.logspace(0.0001, 1, 10), "random_state": range(10)}
    )
    grid.fit(X_train, y_train)
    toc = time.time()
    return grid, tic, toc

In [2]:
import base64
import io

import matplotlib.pyplot as plt
from mandr.infomander import InfoMander
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

for i in range(5):
    X, y = make_classification(random_state=i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    mander = InfoMander(f"probabl-ai/demo-usecase/training/{i}")

    grid, tic, toc = train_model()

    cm = confusion_matrix(y_test, grid.predict(X_test), labels=grid.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid.classes_)
    # disp.plot()

    myio = io.BytesIO()
    plt.savefig(myio, format="jpg")
    myio.seek(0)
    base64_img = base64.b64encode(myio.read()).decode()

    mander.add_view(
        "confusion_matrix", f'<img src="data:image/png;base64, {base64_img}">'
    )
    mander.add_artifact("model", grid)
    mander.add_info("train_time", toc - tic)
    mander.add_info("cv_results", grid.cv_results_)
    mander.add_info("X_shape", list(X.shape))
    mander.add_info("n_classes", len(set(y)))

<Figure size 640x480 with 0 Axes>